## **1.1.2_DATASET_masters_check_if_tassel_vcf_conversion_correct.ipynb**

### GOALS of this script:
* check if the conversion of Unterseer_2016.vcf file into HapMap files is correct by copying rows from the .vcf into the HapMap
* check one row allel column equal to row calls (IUPAC nucleotide code)

In [2]:
import gzip
import allel
import pandas as pd
import numpy as np
import tskit
import tsinfer
import sys
import json
import csv
from IPython.display import SVG
from IPython.display import HTML
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import Workbook
from progressbar import ProgressBar
sns.set_style('white')
sns.set_style('ticks')

## **Read in all dataset files as hapmap format**

### *Read in hapmap Unterseer_2016 files*

In [4]:
unterseer_2016_landraces_hapmap=pd.read_csv("/Users/kschul38/Documents/tsinfer-project/data/2_processed/during_masters_europe_maize_dataset_600k/unterseer_2016_landraces_hapmap.hmp.txt",sep="\t")
unterseer_2016_landraces_hapmap

,rs#,alleles,chrom,pos,strand,assembly#,center,protLSID,assayLSID,panelLSID,...,WA.14,WA.15,WA.16,WA.17,WA.18,WA.19,WA.20,WA.21,WA.22,WA.23
0,ZmSYNBREED_10000_129_2,T/C,1,2179,+,NaN,NaN,NaN,NaN,NaN,...,Y,T,T,T,Y,T,T,T,T,T
1,ZmSYNBREED_10000_164_2,C/T,1,3279,+,NaN,NaN,NaN,NaN,NaN,...,C,T,Y,Y,Y,Y,C,Y,T,T
2,SYN83_2,G/A,1,3498,+,NaN,NaN,NaN,NaN,NaN,...,G,A,R,R,R,R,G,R,A,A
3,ZmSYNBREED_10000_181_2,C/T,1,4140,+,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,C
4,ZmSYNBREED_10000_196_2,C/G,1,4817,+,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615879,PZE-110111453_2,C/T,10,150113255,+,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,C
615880,ZmSYNBREED_23543_720_2,G/A,10,150116206,+,NaN,NaN,NaN,NaN,NaN,...,A,A,R,A,A,A,R,A,R,A
615881,ZmSYNBREED_23545_660_2,G/T,10,150164014,+,NaN,NaN,NaN,NaN,NaN,...,G,G,G,G,G,G,G,G,G,G
615882,ZmSYNBREED_23546_255_2,T/G,10,150172580,+,NaN,NaN,NaN,NaN,NaN,...,T,T,T,T,T,T,T,T,T,T


In [ ]:
unterseer_2016_elite_hapmap=pd.read_csv("/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/unterseer_2016_elite_hapmap.hmp.txt",sep="\t")
unterseer_2016_elite_hapmap

### **Read in duplicate hapmap files**

test_unterseer_for_masters_elite=pd.read_csv("/Users/kschul38/Documents/Tassel_5.0/test_unterseer_for_master_elite.hmp.txt",sep="\t")
test_unterseer_for_masters_elite

test_unterseer_for_masters_landraces=pd.read_csv("/Users/kschul38/Documents/Tassel_5.0/test_unterseer_for_master_landraces.hmp.txt",sep="\t")
test_unterseer_for_masters_landraces

## **Read in all dataset files as .vcf format**

### *Read in Unterseer_2016 files*

In [ ]:
unterseer_2016_landraces_vcf= allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/1_raw/5_Unterseer_2016/TUM-PLANTBREEDING_Maize600k_landraces.vcf',fields='*', log=sys.stdout)

In [ ]:
unterseer_2016_elite_vcf= allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/1_raw/5_Unterseer_2016/TUM-PLANTBREEDING_Maize600k_elitelines.vcf',fields='*', log=sys.stdout)

In [ ]:
sorted(unterseer_2016_landraces_vcf.keys())

In [ ]:
sorted(unterseer_2016_elite_vcf.keys())

## **Get the relevant column from the .vcf file**

### *Get 600k array marker names*

In [ ]:
axiom_marker_unterseer_2016_elite_vcf=unterseer_2016_elite_vcf['variants/AD']
axiom_marker_unterseer_2016_landraces_vcf=unterseer_2016_landraces_vcf['variants/AD']

In [ ]:
print(axiom_marker_unterseer_2016_elite_vcf)
print(len(axiom_marker_unterseer_2016_elite_vcf))
print(axiom_marker_unterseer_2016_landraces_vcf)
print(len(axiom_marker_unterseer_2016_landraces_vcf))

In [ ]:
#compare the marker names between the hapmap files
np.array_equal(axiom_marker_unterseer_2016_elite_vcf,axiom_marker_unterseer_2016_landraces_vcf)

### *Get 600k array marker qualities*

In [ ]:
axiom_quality_marker_unterseer_2016_elite_vcf=unterseer_2016_elite_vcf['variants/CMT']
axiom_quality_marker_unterseer_2016_landraces_vcf=unterseer_2016_landraces_vcf['variants/CMT']

* dosent make sense to compare because marker qualities are determined by the sample

In [ ]:
print(axiom_quality_marker_unterseer_2016_elite_vcf)
print(len(axiom_quality_marker_unterseer_2016_elite_vcf))
print(axiom_quality_marker_unterseer_2016_landraces_vcf)
print(len(axiom_quality_marker_unterseer_2016_landraces_vcf))

### *Get 600k array marker positions*

In [ ]:
pos_unterseer_2016_elite_vcf=unterseer_2016_elite_vcf["variants/POS"]
pos_unterseer_2016_landraces_vcf=unterseer_2016_landraces_vcf["variants/POS"]

In [ ]:
print(pos_unterseer_2016_elite_vcf)
print(len(pos_unterseer_2016_elite_vcf))
print(pos_unterseer_2016_landraces_vcf)
print(len(pos_unterseer_2016_landraces_vcf))

In [ ]:
#compare the columns between Unterseer elite & landrace file
np.array_equal(pos_unterseer_2016_elite_vcf,pos_unterseer_2016_landraces_vcf)

### *Get reference allele column*

In [ ]:
ref_unterseer_2016_elite_vcf=unterseer_2016_elite_vcf["variants/REF"]
ref_unterseer_2016_landraces_vcf=unterseer_2016_landraces_vcf["variants/REF"]

In [ ]:
print(ref_unterseer_2016_elite_vcf)
print(len(ref_unterseer_2016_elite_vcf))
print(ref_unterseer_2016_landraces_vcf)
print(len(ref_unterseer_2016_landraces_vcf))

In [ ]:
#compare the columns between Unterseer elite & landrace files
np.array_equal(ref_unterseer_2016_elite_vcf,ref_unterseer_2016_landraces_vcf)

### *Get alternative allele column*

In [ ]:
alt_unterseer_2016_elite_vcf=unterseer_2016_elite_vcf["variants/ALT"]
alt_unterseer_2016_landraces_vcf=unterseer_2016_landraces_vcf["variants/ALT"]

In [ ]:
print(alt_unterseer_2016_elite_vcf)
print(len(alt_unterseer_2016_elite_vcf))
print(alt_unterseer_2016_landraces_vcf)
print(len(alt_unterseer_2016_landraces_vcf))

In [ ]:
#compare the columns between Unterseer elite & landrace files
np.array_equal(alt_unterseer_2016_elite_vcf,alt_unterseer_2016_landraces_vcf)

### *Get the IDs*

In [ ]:
id_unterseer_2016_elite_vcf=unterseer_2016_elite_vcf["variants/ID"]
id_unterseer_2016_landraces_vcf=unterseer_2016_landraces_vcf["variants/ID"]

In [ ]:
print(id_unterseer_2016_elite_vcf)
print(len(id_unterseer_2016_elite_vcf))
print(id_unterseer_2016_landraces_vcf)
print(len(id_unterseer_2016_landraces_vcf))

In [ ]:
np.array_equal(id_unterseer_2016_elite_vcf,id_unterseer_2016_landraces_vcf)

## **Make the comparisons between .vcf and .hmp.txt**

### *Get the hapmap columns*

In [ ]:
#get the hapmap columns 
print(unterseer_2016_elite_hapmap.columns)
print(unterseer_2016_landraces_hapmap.columns)

### *Get the IDs*

In [ ]:
print(unterseer_2016_elite_hapmap["rs#"])
print(len(unterseer_2016_elite_hapmap["rs#"]))
print(unterseer_2016_landraces_hapmap["rs#"])
print(len(unterseer_2016_landraces_hapmap["rs#"]))

### *Get the positions*

In [ ]:
print(unterseer_2016_elite_hapmap["pos"])
print(len(unterseer_2016_elite_hapmap["pos"]))
print(unterseer_2016_landraces_hapmap["pos"])
print(len(unterseer_2016_landraces_hapmap["pos"]))

In [ ]:
#compare the positions between the hapmap files
np.array_equal(np.array(unterseer_2016_elite_hapmap["pos"]),np.array(unterseer_2016_landraces_hapmap["pos"]))

In [ ]:
#compare the positions between the elite vcf file and the hapmap file 
np.array_equal(pos_unterseer_2016_elite_vcf,np.array(unterseer_2016_elite_hapmap["pos"]))

In [ ]:
#compare the positions between the landraces vcf file and the hapmap file
np.array_equal(pos_unterseer_2016_landraces_vcf,np.array(unterseer_2016_landraces_hapmap["pos"]))

### *Get the ID columns*

In [ ]:
print(unterseer_2016_elite_hapmap["alleles"])
print(len(unterseer_2016_elite_hapmap["alleles"]))
print(unterseer_2016_landraces_hapmap["alleles"])
print(len(unterseer_2016_landraces_hapmap["alleles"]))

In [ ]:
unterseer_2016_landraces_hapmap["alleles"].str.split(pat="/")

In [ ]:
unterseer_2016_elite_hapmap["alleles"].str.split(pat="/")

## **Check suspicios row (last row) of the unterseer_2016_elite_hapmap**

### *Check suspicios row (last row) of the unterseer_2016_elite_hapmap*

* why does the last line of the hapmap elite file just contain A in the allel colum?

In [ ]:
only_IUPAC_calls_unterseer_2016_landraces=unterseer_2016_landraces_hapmap.iloc[:,16:]
only_IUPAC_calls_unterseer_2016_landraces

In [ ]:
last_row_unterseer_2016=only_IUPAC_calls_unterseer_2016_landraces[615883:]

In [ ]:
last_row_unterseer_2016_numpy=last_row_unterseer_2016.to_numpy().flatten()

In [ ]:
pd.unique(last_row_unterseer_2016_numpy) #N = any base, A = Adenine and R= Adenine/Guanine so fine 